# MOPSO FOR CONTROLLER PLACEMENT IN SDWAN WITH FIVE METRICS

In [16]:
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
import random as random
import math

In [17]:
def deleteOneRepositoryMember(rep , gamma):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(gamma*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    #selectedmember = selectedCellmembers[selectedmemberindex]

    # delete memeber
    #rep[selectedmemberindex] = []
    rep = np.delete(rep, selectedmemberindex)

    return rep.tolist()

In [18]:
def SelectLeader(rep , beta):
    gridindices = [item.gridIndex for item in rep]
    OCells = np.unique(gridindices) # ocupied cells
    N = np.zeros(len(OCells))
    for k in range(len(OCells)):
        N[k] = gridindices.count(OCells[k])
    # selection probablity
    p = [math.exp(-beta*item) for item in N]
    p = np.array(p)/sum(p)

    # select cell index
    sci = roulettewheelSelection(p)
    SelectedCell = OCells[sci]

    #selected Cell members
    selectedCellmembers = [item for item in gridindices if item == SelectedCell]

    selectedmemberindex = np.random.randint(0,len(selectedCellmembers))
    # selectedmember = selectedCellmembers[selectedmemberindex]

    return rep[selectedmemberindex]


In [19]:
def roulettewheelSelection(p):
    r = random.random()
    cumsum = np.cumsum(p)
    y = (cumsum<r)
    x= [i for i in y if i==True]
    return len(x)

In [20]:
def FindGridIndex(particle, grid):
    nObj = len(particle.cost)
    NGrid = len(grid[0].LowerBounds)
    
    particle.gridSubIndex = np.zeros((1,nObj))[0]
    for j in range(nObj):  
        index_in_Dim = len( [item for item in grid[j].UpperBounds if particle.cost[j]>item]) 
        particle.gridSubIndex[j] = index_in_Dim

    particle.gridIndex = particle.gridSubIndex[0]

    for j in range(1,nObj):
        particle.gridIndex = particle.gridIndex 
        particle.gridIndex = NGrid*particle.gridIndex
        particle.gridIndex = particle.gridIndex + particle.gridSubIndex[j]

    return particle

In [21]:
def CreateGrid(pop,nGrid,alpha,nobj):
    costs = [item.cost for item in pop]
    Cmin = np.min(costs,axis=0)
    Cmax = np.max(costs,axis=0)
    deltaC = Cmax - Cmin
    Cmin =  Cmin - alpha*deltaC
    Cmax = Cmax + alpha*deltaC
   
    grid = [GridDim() for p in range(nobj)]
    for i in range(nobj):
       dimValues = np.linspace(Cmin[i],Cmax[i],nGrid+1).tolist()
       grid[i].LowerBounds = [-float('inf')] + dimValues
       grid[i].UpperBounds = dimValues  + [float('inf')]
    return grid

In [22]:
def Dominates(x,y):
    x=np.array(x)
    y=np.array(y)
    x_dominate_y = all(x<=y) and any(x<y)
    return x_dominate_y

def DetermineDomination(pop):
    pop_len= len(pop)
    for i in range(pop_len):
         pop[i].IsDominated = False 

    for i in range(pop_len-1):
        for j in range(i+1,pop_len):
            if Dominates(pop[i].cost,pop[j].cost):
                pop[j].IsDominated = True
            if Dominates(pop[j].cost,pop[i].cost):
                pop[i].IsDominated = True

    return pop

In [23]:
def fixer(VA):
    VA = [int(x) for x in VA]
    for i in range(len(VA)):
      if VA[i] < 0:
        VA[i] = 0
      elif VA[i] > 20:
        VA[i] = 20
      
      
    k = []
    for i in range(len(VA)):
        VA[i] = int(round(VA[i]))
        if VA[i] not in k:
            k.append(VA[i])
        else:
            f = 0
            for q in range(VA[i], 20):
                if q not in k:
                    VA[i] = q
                    k.append(q)
                    f = 1
                    break
            if f == 0:
                for q in reversed(range(0, VA[i])):
                    if q not in k:
                        VA[i] = q
                        k.append(q)
                        f = 1
                        break
    return VA

In [24]:
#CUSTOM OBJECTIVE FUNCTION
#edited mocp for list
#CUSTOM OBJECTIVE FUNCTION
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', None)
#creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ)
#creator.create("Individual", list, fitness=creator.FitnessMin)
import math
#Introduction of repairs operator/local mutation to improve on NSGA-111 code.
def uniform(low, up, size=None):
  return random.sample(range(varMin,varMax),nVar)
import autograd.numpy as anp        
def nuNumber(dup, row, ct = 1):    #dup=duplicate, row=row, and the ct=control variable.
                  #print('duplicated number is ', dup, ' in ', row, ' iterator is ', ct)
                  if dup < 20:     #setting check condition to the maximum number of BtEurope
                    nu = dup + ct
                  else:
                    nu = dup - ct
                  if nu < 1:
                    nuNumber(nu + 1, row, ct + 1)
                  if nu in row:
                    nu = nuNumber(nu + 1, row, ct + 1)
                  
                  if (math.isnan(nu)):   #when nan appears in row, then replace with random from q gen
                      for q in range(0,20):
                        if(q not in row ):
                          nu = q
                          break
                  return round(nu)

def MOCPParticle(nVar): #passing VARS=init pop and NDIM=number of decision variables into the funct.
        #print(type(VARS))
        #if isinstance(nVar,np.ndarray):
           # nVar = nVar.tolist()
        #if type(nVar) is numpy.ndarray:
        if not isinstance(nVar, list):
            nVar = nVar.tolist()
            nVar = [int(x) for x in nVar]
       
       
        #nVar = list(nVar)
        #if isinstance(nVar,np.ndarray):
         #   nVar = type(nVar)
          #  return 0
                
        #print("Details of X below")

        #VARS.apply(pd.to_numeric)
        #VARS.astype('int32').dtypes
        #df = VARS.convert_objects(convert_numeric=True)
        dataset = pd.read_csv('DistanceMatrixnan2.csv',index_col=0)
        global x
        #x = pd.DataFrame(nVar)
        
        li = []
        cont_distance =[]
        cont_distanc =[]
        ctyu = []
        sam = 0
        a = 0
        sm = []
        city = dataset.columns.to_list()
        #print("city is \n", city)
        v = len(dataset)
        #print("length of dataset is ", v)
        minL = []
        sumL = []
        maxC2C = []
        avgC2C = []
        C2C = []
        end_values = []
        
        r = nVar
        print (r)
        for i in range(len(r)):
            ctyu.append(city[r[i]])
          #print('city being appended is ',city[r[i]])

        
          #print('selected', city[r[i]]  )
        a = i
        dty = dataset[ctyu].iloc[0]

          #print('tobi is ',dty)
        cont_distance = max(dty)
          
        ydt = dataset[ctyu]
        lo = ydt.iloc[0]
        #print(lo)
        for i in range(len(lo)):
            si = 0
            a = i
              
            for j in range(i +1,len(lo), 1):
                si = lo[i] + lo[j]
                sm.append(si)
            t = 0
            for u in sm:
              t = t + u
            #print(t)
            C2 = (t)/(math.comb(len(r), 2))
            print(C2)
        new_dataset = ydt.dropna() 
        
        row_list = []
        new_list =[]
        ij = []
        values = pd.DataFrame()
        for index, row in ydt.iterrows():
            #print('index is length ',len(row))
            for k in range(len(row)):
              
              if isinstance(row.loc[ctyu[k]], pd.core.series.Series):
                
                row_list.append(row.loc[ctyu[k]].iloc[0])
              else:
                row_list.append(row.loc[ctyu[k]])
            #print("row_list\n", row_list)
            for i in row_list:
              if i != min(row_list):
                    #print("I is not equal to min of rowlist")
                    new_list.append(0)
              else:
                    new_list.append(1)
            #print("New list is \n",new_list)
            to_load = pd.Series(new_list, index=ctyu)
            #print("loaded list is \n", to_load)
            values = values.append(to_load, ignore_index=True)
            #print("values \n", values)
            row_list =[]
            new_list =[]

            result = values.apply(pd.value_counts).drop([0])
            #print("result\n", result,"\nMAX", result.max(axis = 1), "\nMIN", result.min(axis = 1))
            end_val = (result.max(axis = 1) - result.min(axis = 1))/result.max(axis = 1)
            
            #print("end_val\n",end_val)
            end_values.append(end_val)

            
            
            
            C2C.append(C2)
            maxC2C.append(cont_distance)
            avgC2C.append(C2)

            for i in r:
              li.append(city[i])
            #print("A new row ") 
            #print(li)
            new_dataset2 = dataset[li]  
            new_dataset2 = new_dataset2.dropna() 
            new_dataset2 = new_dataset2.reset_index(drop=True)
            new_dataset2 =  pd.DataFrame(new_dataset2)
            mins =  new_dataset2.min(axis=1)
            sum = 1/(abs(len(new_dataset2))) * np.sum(mins)
            #print(new_dataset2)
            minL.append(mins)
            sumL.append(sum)
        minDataset = pd.DataFrame(minL)
        maxC2CDataset = pd.DataFrame(maxC2C)
        C2CDataset = pd.DataFrame(C2C)
        EndValDataset = pd.DataFrame(end_values)
        avgC2CDataset = pd.DataFrame(avgC2C)
        
        #norm_EndValDataset = EndValDataset / (EndValDataset.max())
        #print('EndValDataset.max() is \n', EndValDataset.max())
        #print('EndValDataset is \n',norm_EndValDataset)
        #return norm_EndValDataset
        #endvalues_norm = norm_EndValDataset[1.0].to_list()
        
        maxNu = minDataset.max(axis=1) #generates the maximum of minimum
        maxIjogbon = maxNu.values.tolist()
        end_valmod = end_val.values.tolist()
        #print("Maximum of Minimum is ")
        #print(maxNu)
        #print("\n\n")
        #print("Sum")
        maxNuDataset = pd.DataFrame(maxNu)
        sumDataset = pd.DataFrame(sumL) #generates the sum
        
        print("maxIjogbon", maxIjogbon[0])
        print("cont_distance",  cont_distance)
        print("C2", C2 )
        print("end_valmod", end_valmod[0])
        print("sum", sum)
        
        return numpy.array([maxIjogbon[0], cont_distance, C2, end_valmod[0],  sum])
        #print ("maxNu", maxNu, "\nmaxC2C", maxC2C, "\nCSC",C2C, "\nend_values",   end_values, "\nsumDataset", sumL   )
        #f = anp.column_stack([maxNu, maxC2C,C2C, end_values, sumL ])
        #print("Our F\n", f)
        #return f
#costfunction = lambda nVar: MOCP(nVar)
   
        #out["G"] = anp.column_stack([g1, g2])
#lj = [14,	8,	5,	0,	20]
#MOCPParticle(lj)

In [11]:
#nVar = 5 # number of decision vars
#varMin = -4
#varMax = 4
#maxIt = 100
#nPop = 200    # population size
#nRep = 50  # size of repository
#w = 0.5 # inertia wieght
#c1 = 2 # personal learning coefficient
#c2 = 2 # global learning coefficient
#wdamping = 0.99


In [25]:
#CUSTOM INITIAL PARAMETER TOBESKY
nVar = 5 # number of decision vars
varMin = 0
varMax = 20
maxIt = 100
nPop = 200
nRep = 50  # size of repository
w = 0.5 # inertia wieght
c1 = 5 # personal learning coefficient
c2 = 5 # global learning coefficient
wdamping = 0.99

In [26]:
D_lower = varMin
D_upper = varMax
NDIM = nVar
def uniform(low, up, size=None):
    return random.sample(range(D_lower,D_upper), NDIM)
uniform(D_lower,D_upper, NDIM)

[11, 7, 14, 6, 4]

In [339]:
# ################ constriction coefficients
# phi1 = 2.05
# phi2 = 2.05
# phi = phi1+phi2
# chi = 2/(phi - 2 + np.sqrt(phi**2 - 4*phi))
# w = chi # inertia wieght
# c1 = chi*phi1 # personal learning coefficient
# c2 = chi*phi2 # global learning coefficient
# wdamping = 1
# #################

beta = 1 # leader selection pressure
gamma = 1 # deletion selection pressure
NoGrid = 5
alpha=0.1 # nerkhe tavarrom grid

# initialization
class Particle:
    position = []
    cost = []
    velocity = []
    best_position = []
    best_cost = []
    IsDominated = []
    gridIndex = []
    gridSubIndex = []

# for each objective a grid items is division of values of objective cost
class GridDim: 
    LowerBounds = []
    UpperBounds = []

global Particles
#Particles = np.matlib.repmat(Particle,nPop,1)
Particles = [Particle() for p in range(nPop)]
Temp_Particles = []
#print (Particles)

for i in range(nPop):
    Temp_Position = np.array(uniform(varMin,varMax,nVar))
    Particles[i].position = Temp_Position
    Temp_Particles.append(Temp_Position)
    #print('position is', Particles[i].position)
    Particles[i].velocity = np.zeros(nVar)
    Particles[i].cost = np.array(MOCPParticle(Particles[i].position))
    # update best personal Best
    Particles[i].best_position = Particles[i].position
    Particles[i].best_cost = Particles[i].cost
    Particles[i].IsDominated = False 

print (Particles)
#Particles.cost = MOCP(Temp_Particles, NDIM)   
    
Particles = DetermineDomination(Particles)

Repos = [item for item in Particles if item.IsDominated == False ]
print("Repos\n\n")
print (Repos[0].cost)
print (Repos[1].cost)
nObj =len( Repos[0].cost)
grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)

for r in range(len(Repos)):
    Repos[r] = FindGridIndex(Repos[0],grid)
    #print("r is", r)
    #print ("repos r is", r)


[15, 17, 4, 7, 5]
0.3642725822
0.5820881871999999
0.7182071604999999
0.7879480855999998
0.7879480855999998
maxIjogbon 0.509767536
cont_distance 0.557618536
C2 0.7879480855999998
end_valmod 0.7142857142857143
sum 0.158609485
[16, 15, 9, 11, 8]
0.45738600809999996
0.7691522602
0.9644555912999999
1.0581136428
1.0581136428
maxIjogbon 0.653394307
cont_distance 0.669677499
C2 1.0581136428
end_valmod 0.9
sum 0.18596255185714283
[16, 8, 14, 3, 9]
0.39108037949999996
0.5783978989999999
0.734803653
0.7928911284
0.7928911284
maxIjogbon 0.488235886
cont_distance 0.642858658
C2 0.7928911284
end_valmod 0.7142857142857143
sum 0.1407757063333333
[9, 12, 4, 8, 13]
0.3470337717
0.5898173298
0.7172293936
0.7782634092
0.7782634092
maxIjogbon 0.310969875
cont_distance 0.508226398
C2 0.7782634092
end_valmod 0.8
sum 0.1216491488095238
[2, 7, 16, 10, 13]
0.2618611928
0.5478665402
0.7715292680999999
0.8666202643999998
0.8666202643999998
maxIjogbon 0.357032701
cont_distance 0.642858658
C2 0.8666202643999998
end

maxIjogbon 0.488235886
cont_distance 0.669677499
C2 0.6003281055999999
end_valmod 0.8888888888888888
sum 0.14509859447619045
[6, 15, 0, 1, 3]
0.18908624170000002
0.3826894134
0.4090070243
0.43532463520000003
0.43532463520000003
maxIjogbon 0.488235886
cont_distance 0.557618536
C2 0.43532463520000003
end_valmod 0.8571428571428571
sum 0.16389865580952379
[18, 13, 14, 4, 16]
0.336483904
0.5861490752
0.7819422437
0.8794171336
0.8794171336
maxIjogbon 0.428657641
cont_distance 0.642858658
C2 0.8794171336
end_valmod 0.75
sum 0.14864642828571428
[15, 6, 8, 17, 1]
0.3338374042
0.4981307826
0.6088593797
0.6662073736
0.6662073736
maxIjogbon 0.539145578
cont_distance 0.557618536
C2 0.6662073736
end_valmod 0.8571428571428571
sum 0.1507837842857143
[0, 8, 15, 7, 11]
0.1916294002
0.4366394036
0.6573403557999999
0.7665176008
0.7665176008
maxIjogbon 0.495201728
cont_distance 0.669677499
C2 0.7665176008
end_valmod 0.6666666666666666
sum 0.14508391823809524
[16, 18, 4, 5, 8]
0.3834307591
0.5874711354
0.70

cont_distance 0.669677499
C2 0.7986543676
end_valmod 0.7142857142857143
sum 0.10198413128571428
[15, 10, 16, 0, 17]
0.38637578110000004
0.6711987124000001
0.8380656626
0.8763608812000001
0.8763608812000001
maxIjogbon 0.495201728
cont_distance 0.642858658
C2 0.8763608812000001
end_valmod 0.9090909090909091
sum 0.163041721
[6, 2, 1, 18, 3]
0.1872696372
0.297669915
0.3819168414
0.4280582172
0.4280582172
maxIjogbon 0.44158265
cont_distance 0.388765402
C2 0.4280582172
end_valmod 0.8
sum 0.15694002033333332
[11, 10, 7, 6, 19]
0.4393069907999999
0.7322375466000001
0.8851357506000002
0.9536149644000002
0.9536149644000002
maxIjogbon 0.268416266
cont_distance 0.669677499
C2 0.9536149644000002
end_valmod 0.7142857142857143
sum 0.10809282885714286
[9, 11, 19, 4, 5]
0.3727062827
0.676057506
0.8202329991000001
0.8809534532000001
0.8809534532000001
maxIjogbon 0.310969875
cont_distance 0.669677499
C2 0.8809534532000001
end_valmod 0.875
sum 0.130099319
[2, 15, 1, 0, 12]
0.1846096373
0.4204681462
0.5080

maxIjogbon 0.403059166
cont_distance 0.417275195
C2 0.5728615791999999
end_valmod 0.8888888888888888
sum 0.12932662204761905
[0, 18, 14, 17, 4]
0.1595199222
0.3967929248
0.5665954461
0.6380796888
0.6380796888
maxIjogbon 0.495201728
cont_distance 0.491591393
C2 0.6380796888
end_valmod 0.4
sum 0.16209341295238097
[11, 7, 12, 6, 19]
0.42807988129999996
0.6727077531999999
0.8402273125999999
0.9087065263999999
0.9087065263999999
maxIjogbon 0.267516943
cont_distance 0.669677499
C2 0.9087065263999999
end_valmod 0.75
sum 0.11473465166666667
[17, 16, 9, 1, 0]
0.2873421553
0.5500751678
0.6707732227
0.6898259979999999
0.6898259979999999
maxIjogbon 0.495201728
cont_distance 0.642858658
C2 0.6898259979999999
end_valmod 0.75
sum 0.12768597885714286
[19, 0, 7, 15, 17]
0.3031766453
0.43944321259999997
0.617919275
0.7119763472
0.7119763472
maxIjogbon 0.357032701
cont_distance 0.557618536
C2 0.7119763472
end_valmod 0.2
sum 0.114013687
[7, 6, 1, 0, 14]
0.26380158930000003
0.41226858680000006
0.4995332767

maxIjogbon 0.299491969
cont_distance 0.669677499
C2 0.9585323211999999
end_valmod 0.9090909090909091
sum 0.11939147666666665
[19, 18, 6, 2, 3]
0.254871857
0.4205867163999999
0.4964236494999999
0.5187571939999999
0.5187571939999999
maxIjogbon 0.423034124
cont_distance 0.417275195
C2 0.5187571939999999
end_valmod 0.875
sum 0.12366459795238097
[4, 14, 1, 2, 12]
0.2655568927
0.4966759676
0.5993703999
0.6639592816000001
0.6639592816000001
maxIjogbon 0.428657641
cont_distance 0.495201728
C2 0.6639592816000001
end_valmod 0.5
sum 0.11877824299999999
[6, 13, 16, 15, 8]
0.2880885122
0.5378576751999999
0.7488815619999999
0.8313337171999999
0.8313337171999999
maxIjogbon 0.403059166
cont_distance 0.642858658
C2 0.8313337171999999
end_valmod 0.875
sum 0.13618095238095237
[4, 0, 3, 18, 14]
0.2280566115
0.32335712660000004
0.4259224773
0.5139581568
0.5139581568
maxIjogbon 0.488235886
cont_distance 0.491591393
C2 0.5139581568
end_valmod 0.8571428571428571
sum 0.15773311871428575
[4, 5, 9, 11, 15]
0.333

maxIjogbon 0.252448085
cont_distance 0.557618536
C2 0.8561359268000001
end_valmod 0.7142857142857143
sum 0.09542237490476191
[2, 15, 16, 6, 17]
0.24536946789999997
0.5419878074
0.7356064519000001
0.8006533647999999
0.8006533647999999
maxIjogbon 0.44158265
cont_distance 0.642858658
C2 0.8006533647999999
end_valmod 0.8571428571428571
sum 0.13698518661904763
[4, 12, 9, 14, 13]
0.31660176230000003
0.5994877738000001
0.7846359066999999
0.8681387600000001
0.8681387600000001
maxIjogbon 0.331890241
cont_distance 0.508226398
C2 0.8681387600000001
end_valmod 0.8
sum 0.11242546357142857
[12, 2, 19, 5, 16]
0.34669421540000006
0.5254451574000001
0.7007174922000001
0.792534788
0.792534788
maxIjogbon 0.490670485
cont_distance 0.642858658
C2 0.792534788
end_valmod 0.7777777777777778
sum 0.12014813866666665
[9, 8, 16, 19, 14]
0.3851533854
0.6203968148
0.8398552051999999
0.930741864
0.930741864
maxIjogbon 0.268416266
cont_distance 0.642858658
C2 0.930741864
end_valmod 0.7142857142857143
sum 0.1152516129

Repos


[0.36554314 0.60747282 0.65820232 0.71428571 0.11688778]
[0.44158265 0.38295219 0.5633443  0.85714286 0.15743495]


In [ ]:
numpy.array(Particles[10].best_position)
print (type(numpy.array(Particles[10].best_position)))
if type(Particles[10].best_position) is numpy.ndarray:
    print("isarray")
else:
    print("notarray")

In [335]:
numpy.array(Particles[10].velocity)

                
    
    
      

array([0., 0., 0., 0., 0.])

In [347]:
# MOPSO main loop
for it in range(maxIt):
    for i in range(nPop):
        leader = SelectLeader(Repos,beta)
        print ("leader", leader)
        # update velocity
        print ("Particles[i].best_position", Particles[i].best_position, "type", 
               type(Particles[i].best_position))
        leader.position = np.array(leader)

        #print ("Repos", Repos, "type", type(Repos))
        #print ("beta", beta, "type", type(beta))
        #print ("Particles[i].velocity", Particles[i].velocity, "type", type(Particles[i].velocity))
        
       # Particles[i].position = np.array(Particle[i].position)
        
        #print ("Particles[i].position", Particles[i].position, "type", 
         #      type(Particles[i].position))
        #print ("leader.position", leader.position, "type", 
          #     type(leader.position))
        
        Particles[i].velocity = w*Particles[i].velocity  \
            + c1*np.random.rand(1,nVar)[0]*(Particles[i].best_position - Particles[i].position) \
            + c2*np.random.rand(1,nVar)[0]*(leader.position - Particles[i].position)
        
       
              
        # update position
        Particles[i].position = Particles[i].position + Particles[i].velocity 

        # evaluation
        tp = fixer(Particles[i].position)
        Particles[i].position = tp
        Particles[i].cost = MOCPParticle(Particles[i].position)
        

        if Dominates(Particles[i].cost,Particles[i].best_cost):
            Particles[i].best_position = Particles[i].position
            Particles[i].best_cost = Particles[i].cost
        else:
            if np.random.rand() > 0.5:
                Particles[i].best_position = Particles[i].position
                Particles[i].best_cost = Particles[i].cost
      
   
    Repos = Repos + Particles
    Repos = DetermineDomination(Repos)
    Repos = [item for item in Repos if item.IsDominated == False ]

    grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)
    for r in range(len(Repos)):
        Repos[r] = FindGridIndex(Repos[r],grid)

    # check if repository is full
    if len(Repos) > nRep :
        extra = len(Repos) - nRep
        for e in range(extra):
            Repos = deleteOneRepositoryMember(Repos,gamma)

    ########## show figure ########## 
    plt.clf()
    particlesCost = np.reshape( [item.cost for item in Particles ],newshape=(nPop,5))
    repositoryCost = [item.cost for item in Repos]
    repositoryCost = np.reshape( repositoryCost, newshape=(len(repositoryCost),5))
    plt.plot(particlesCost[:,0], particlesCost[:,1], 'o' ,mfc='none')
    plt.plot(repositoryCost[:,0], repositoryCost[:,1], 'r*')
    
    plt.draw()
    plt.pause(0.00000000001)

    w=w*wdamping
    
    # print(repositoryCost)
    # print("ok")
    # print(particlesCost)
    ########## show figure ##########
plt.show()

leader <__main__.Particle object at 0x000001FD383E9D30>
Particles[i].best_position [15 17  4  7  5] type <class 'numpy.ndarray'>


TypeError: unsupported operand type(s) for -: 'Particle' and 'int'

# TESTING OF MAIN LOOP

In [346]:
# MOPSO main loop
for it in range(maxIt):
    for i in range(nPop):
        leader = SelectLeader(Repos,beta)
        print ("leader", leader)
        # update velocity
        print ("Particles[i].best_position", Particles[i].best_position, "type", 
               type(Particles[i].best_position))
        leader.position = np.array(leader)

        #print ("Repos", Repos, "type", type(Repos))
        #print ("beta", beta, "type", type(beta))
        #print ("Particles[i].velocity", Particles[i].velocity, "type", type(Particles[i].velocity))
        
       # Particles[i].position = np.array(Particle[i].position)
        
        #print ("Particles[i].position", Particles[i].position, "type", 
         #      type(Particles[i].position))
        #print ("leader.position", leader.position, "type", 
          #     type(leader.position))
        
        Particles[i].velocity = w*Particles[i].velocity  \
            + c1*np.array(uniform(varMin,varMax,nVar))[0]*(Particles[i].best_position - Particles[i].position) \
            + c2*np.array(uniform(varMin,varMax,nVar))[0]*(leader.position - Particles[i].position)
        
       
              
        # update position
        Particles[i].position = Particles[i].position + Particles[i].velocity 

        # evaluation
        tp = fixer(Particles[i].position)
        Particles[i].position = tp
        Particles[i].cost = MOCPParticle(Particles[i].position)
        

        if Dominates(Particles[i].cost,Particles[i].best_cost):
            Particles[i].best_position = Particles[i].position
            Particles[i].best_cost = Particles[i].cost
        else:
            if np.random.rand() > 0.5:
                Particles[i].best_position = Particles[i].position
                Particles[i].best_cost = Particles[i].cost
      
   
    Repos = Repos + Particles
    Repos = DetermineDomination(Repos)
    Repos = [item for item in Repos if item.IsDominated == False ]

    grid = CreateGrid(Repos,NoGrid,alpha=0.1,nobj=nObj)
    for r in range(len(Repos)):
        Repos[r] = FindGridIndex(Repos[r],grid)

    # check if repository is full
    if len(Repos) > nRep :
        extra = len(Repos) - nRep
        for e in range(extra):
            Repos = deleteOneRepositoryMember(Repos,gamma)

    ########## show figure ########## 
    plt.clf()
    particlesCost = np.reshape( [item.cost for item in Particles ],newshape=(nPop,5))
    repositoryCost = [item.cost for item in Repos]
    repositoryCost = np.reshape( repositoryCost, newshape=(len(repositoryCost),5))
    plt.plot(particlesCost[:,0], particlesCost[:,1], 'o' ,mfc='none')
    plt.plot(repositoryCost[:,0], repositoryCost[:,1], 'r*')
    
    plt.draw()
    plt.pause(0.00000000001)

    w=w*wdamping
    
    # print(repositoryCost)
    # print("ok")
    # print(particlesCost)
    ########## show figure ##########
plt.show()

leader <__main__.Particle object at 0x000001FD383E9D30>
Particles[i].best_position [15 17  4  7  5] type <class 'numpy.ndarray'>


TypeError: unsupported operand type(s) for -: 'Particle' and 'int'

In [ ]:
gridindices = [item.gridIndex for item in Repos]
gridindices

# END OF WORKING MOPSO

In [ ]:
Particles[0].position


In [ ]:
#edited mocp for list
#CUSTOM OBJECTIVE FUNCTION
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', None)
#creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ)
#creator.create("Individual", list, fitness=creator.FitnessMin)
import math
#Introduction of repairs operator/local mutation to improve on NSGA-111 code.
def uniform(low, up, size=None):
  return random.sample(range(varMin,varMax),nVar)
import autograd.numpy as anp        
def nuNumber(dup, row, ct = 1):    #dup=duplicate, row=row, and the ct=control variable.
                  #print('duplicated number is ', dup, ' in ', row, ' iterator is ', ct)
                  if dup < 20:     #setting check condition to the maximum number of BtEurope
                    nu = dup + ct
                  else:
                    nu = dup - ct
                  if nu < 1:
                    nuNumber(nu + 1, row, ct + 1)
                  if nu in row:
                    nu = nuNumber(nu + 1, row, ct + 1)
                  
                  if (math.isnan(nu)):   #when nan appears in row, then replace with random from q gen
                      for q in range(0,20):
                        if(q not in row ):
                          nu = q
                          break
                  return round(nu)

def MOCPParticle(nVar): #passing VARS=init pop and NDIM=number of decision variables into the funct.
        #print(type(VARS))
       
        
        #print("Details of X below")

        #VARS.apply(pd.to_numeric)
        #VARS.astype('int32').dtypes
        #df = VARS.convert_objects(convert_numeric=True)
        dataset = pd.read_csv('DistanceMatrixnan2.csv',index_col=0)
        global x
        #x = pd.DataFrame(nVar)
        
        li = []
        cont_distance =[]
        cont_distanc =[]
        ctyu = []
        sam = 0
        a = 0
        sm = []
        city = dataset.columns.to_list()
        #print("city is \n", city)
        v = len(dataset)
        #print("length of dataset is ", v)
        minL = []
        sumL = []
        maxC2C = []
        avgC2C = []
        C2C = []
        end_values = []
        
        r = nVar
        for i in range(len(r)):
            ctyu.append(city[r[i]])
          #print('city being appended is ',city[r[i]])

        
          #print('selected', city[r[i]]  )
        a = i
        dty = dataset[ctyu].iloc[0]

          #print('tobi is ',dty)
        cont_distance = max(dty)
          
        ydt = dataset[ctyu]
        lo = ydt.iloc[0]
        #print(lo)
        for i in range(len(lo)):
            si = 0
            a = i
              
            for j in range(i +1,len(lo), 1):
                si = lo[i] + lo[j]
                sm.append(si)
            t = 0
            for u in sm:
              t = t + u
            #print(t)
            C2 = (t)/(math.comb(len(r), 2))
            print(C2)
        new_dataset = ydt.dropna() 
        
        row_list = []
        new_list =[]
        ij = []
        values = pd.DataFrame()
        for index, row in ydt.iterrows():
            #print('index is length ',len(row))
            for k in range(len(row)):
              
              if isinstance(row.loc[ctyu[k]], pd.core.series.Series):
                
                row_list.append(row.loc[ctyu[k]].iloc[0])
              else:
                row_list.append(row.loc[ctyu[k]])
            #print("row_list\n", row_list)
            for i in row_list:
              if i != min(row_list):
                    #print("I is not equal to min of rowlist")
                    new_list.append(0)
              else:
                    new_list.append(1)
            #print("New list is \n",new_list)
            to_load = pd.Series(new_list, index=ctyu)
            #print("loaded list is \n", to_load)
            values = values.append(to_load, ignore_index=True)
            #print("values \n", values)
            row_list =[]
            new_list =[]

            result = values.apply(pd.value_counts).drop([0])
            #print("result\n", result,"\nMAX", result.max(axis = 1), "\nMIN", result.min(axis = 1))
            end_val = (result.max(axis = 1) - result.min(axis = 1))/result.max(axis = 1)
            
            #print("end_val\n",end_val)
            end_values.append(end_val)

            
            
            
            C2C.append(C2)
            maxC2C.append(cont_distance)
            avgC2C.append(C2)

            for i in r:
              li.append(city[i])
            #print("A new row ") 
            #print(li)
            new_dataset2 = dataset[li]  
            new_dataset2 = new_dataset2.dropna() 
            new_dataset2 = new_dataset2.reset_index(drop=True)
            new_dataset2 =  pd.DataFrame(new_dataset2)
            mins =  new_dataset2.min(axis=1)
            sum = 1/(abs(len(new_dataset2))) * np.sum(mins)
            #print(new_dataset2)
            minL.append(mins)
            sumL.append(sum)
        minDataset = pd.DataFrame(minL)
        maxC2CDataset = pd.DataFrame(maxC2C)
        C2CDataset = pd.DataFrame(C2C)
        EndValDataset = pd.DataFrame(end_values)
        avgC2CDataset = pd.DataFrame(avgC2C)
        
        #norm_EndValDataset = EndValDataset / (EndValDataset.max())
        #print('EndValDataset.max() is \n', EndValDataset.max())
        #print('EndValDataset is \n',norm_EndValDataset)
        #return norm_EndValDataset
        #endvalues_norm = norm_EndValDataset[1.0].to_list()
        
        maxNu = minDataset.max(axis=1) #generates the maximum of minimum
        maxIjogbon = maxNu.values.tolist()
        end_valmod = end_val.values.tolist()
        #print("Maximum of Minimum is ")
        #print(maxNu)
        #print("\n\n")
        #print("Sum")
        maxNuDataset = pd.DataFrame(maxNu)
        sumDataset = pd.DataFrame(sumL) #generates the sum
        
        print("maxIjogbon", maxIjogbon[0])
        print("cont_distance",  cont_distance)
        print("C2", C2 )
        print("end_valmod", end_valmod[0])
        print("sum", sum)
        
        return [maxIjogbon[0], cont_distance, C2, end_valmod[0],  sum]
        #print ("maxNu", maxNu, "\nmaxC2C", maxC2C, "\nCSC",C2C, "\nend_values",   end_values, "\nsumDataset", sumL   )
        f = anp.column_stack([maxNu, maxC2C,C2C, end_values, sumL ])
        #print("Our F\n", f)
        return f
#costfunction = lambda nVar: MOCP(nVar)
   
        #out["G"] = anp.column_stack([g1, g2])
lj = [14,	8,	5,	0,	20]
MOCPParticle(lj)

# MOPSO TEMPLATE CODE3 FOR CONTROLLER PLACEMENT

In [ ]:
# FUNCTION TO CHECK NON DOMINATION OF PARTICLES IN POPULATION AND ARCHIVE

import random
from math import *
import numpy
import copy



def check_nondom(i,archive_temp,no_of_obj_func,ARCHIVE_FIT,PARTICLE_FIT,COUNT,ARCHIVE_VAL):
    h=0
    while(h<=int(archive_temp[0])):
        summ = 0;
        if((ARCHIVE_FIT[h][0]<PARTICLE_FIT[i][0]) and (ARCHIVE_FIT[h][1]<PARTICLE_FIT[i][1]) and (ARCHIVE_FIT[h][2]>PARTICLE_FIT[i][2])):
            summ+=no_of_obj_func   # if particle in archive dominates
            return(0)
        elif((ARCHIVE_FIT[h][0]>=PARTICLE_FIT[i][0]) and (ARCHIVE_FIT[h][1]>=PARTICLE_FIT[i][1]) and (ARCHIVE_FIT[h][2]<=PARTICLE_FIT[i][2])):
            summ=0     # if particle in archive is dominated , delete it
            for j in range(COUNT):
                ARCHIVE_VAL[h][j]=ARCHIVE_VAL[int(archive_temp[0])-1][j]
            for j in range(no_of_obj_func):
                ARCHIVE_FIT[h][j]=ARCHIVE_FIT[int(archive_temp[0])-1][j]
            archive_temp[0]-=1
        else:
            h+=1
            
    return(1)

In [ ]:
# COMPUTE CROWDING DISTANCES OF PARTICLES IN ARCHIVE

import nbimporter
import random
from math import *
import copy
import numpy
import FITNESS_SORT
import DISTANCE_COMPUTATION
import CROWD_SORT
reload(CROWD_SORT)
reload(DISTANCE_COMPUTATION)
reload(FITNESS_SORT)


def crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL):
    CrowdList=[float(0.0) for i in range(no_of_nondom_sol)]   #initialise crowding distance values
    # objective function 1
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(0,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,1)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(0,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,1)
    # objective function 2
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(1,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,1)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(1,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,1)
    # objective function 3
    begin=0
    # sort fitness values
    no_of_nondom_sol=FITNESS_SORT.sort_fitness(2,begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,2)
    # compute crowding distance
    no_of_nondom_sol=DISTANCE_COMPUTATION.compute_distance(2,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,2)
    
    begin=0
    # sort crowding distance values
    no_of_nondom_sol=CROWD_SORT.sort_crowd(begin,no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)
    return(no_of_nondom_sol)

In [ ]:
import random
from math import *
import copy
import numpy
import nbimporter
import CROWD_SORT
reload(CROWD_SORT)


def sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList):
    l=begin+1
    r=lastpart
    turn=CrowdList[begin]
    flagF=[[] for i in range(no_of_obj_func)]
    flagP=[[] for i in range(COUNT)]
    while(l<r):
        if(CrowdList[l]<=turn):
            l+=1
        else:
            r-=1
            # swap the crowding distance values
            temp=CrowdList[l]
            CrowdList[l]=CrowdList[r]
            CrowdList[r]=temp
            # swap fitness values of two particles in the archive
            for k in range(no_of_obj_func):
                flagF[k]=ARCHIVE_FIT[l][k]
                ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                ARCHIVE_FIT[r][k]=flagF[k]
            # swap position values of two particles in the archive    
            for k in range(COUNT):
                flagP[k]=ARCHIVE_VAL[l][k]
                ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                ARCHIVE_VAL[r][k]=flagP[k]
           
    return(lastpart)
#     #swap fitness values of two particles in the archive
#     for k in range(no_of_obj_func):
#         flagF[k]=ARCHIVE_FIT[begin][k]
#         ARCHIVE_FIT[begin][k]=ARCHIVE_FIT[l][k]
#         ARCHIVE_FIT[l][k]=flagF[k]
#      # swap position values of two particles in the archive 
#     for k in range(COUNT):
#         flagP[k]=ARCHIVE_VAL[begin][k]
#         ARCHIVE_VAL[begin][k]=ARCHIVE_VAL[l][k]
#         ARCHIVE_VAL[l][k]=flagP[k]
#     # swap their crowding distance values
#     temp = CrowdList[begin];
#     CrowdList[begin] = CrowdList[l];
#     CrowdList[l] = temp;
    
#     if((l-begin)>1):
#         lastpart=CROWD_SORT.sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)
#     if((lastpart-r)>1):
#         lastpart=CROWD_SORT.sort_crowd(begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList)

In [ ]:
# FUNCTION TO DELETE PARTICLE IN ARCHIVE

import random
from math import *
import copy
import numpy


def delete_particle(k,no_of_obj_func,no_of_nondom_sol,ARCHIVE_FIT):
    # if infeasible particle is the last in archive or only one particle in archive
    if((no_of_nondom_sol== 1) | (k == no_of_nondom_sol-1)):
        no_of_nondom_sol-=1
    # move last particle in archive in place of infeasible particle
    else:
        for j in range(no_of_obj_func):
            ARCHIVE_FIT[k][j]=ARCHIVE_FIT[no_of_nondom_sol-1][j]
        no_of_nondom_sol-= 1;
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO COMPUTE CROWDING DISTANCE NON DOMINATED PARTICLES

import random
from math import *
import copy

def compute_distance(f,no_of_nondom_sol,CrowdList,ARCHIVE_FIT,am):
    maximum=1
    minimum=1
    if(am==1):
        for i in range(1,no_of_nondom_sol-1):
            print type(ARCHIVE_FIT[i][f][0])
            CrowdList[i]=CrowdList[i]+((ARCHIVE_FIT[i+1][f])-(ARCHIVE_FIT[i-1][f]))
            if(CrowdList[minimum]>CrowdList[i]):
                minimum=i
        CrowdList[0]=CrowdList[0]+CrowdList[minimum]
        CrowdList[no_of_nondom_sol-1]=CrowdList[no_of_nondom_sol-1]+CrowdList[minimum]
        
    elif(am==2):
        for i in range(1,no_of_nondom_sol-1):
            print type(ARCHIVE_FIT[i][f])
            CrowdList[i]=CrowdList[i]+((ARCHIVE_FIT[i+1][f])-(ARCHIVE_FIT[i-1][f]))
            if(CrowdList[maximum]<CrowdList[i]):
                maximum=i
        CrowdList[0]=CrowdList[0]+CrowdList[maximum]
        CrowdList[no_of_nondom_sol-1]=CrowdList[no_of_nondom_sol-1]+CrowdList[maximum]
    
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO SORT FITNESS VALUES OF PARTICLES IN ARCHIVE

import random
from math import *
import copy
import numpy
import nbimporter
import FITNESS_SORT
reload(FITNESS_SORT)


def sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am):
    l=begin+1
    r=lastpart
    turn=ARCHIVE_FIT[begin][f]
    flagF=[[] for i in range(no_of_obj_func)]
    flagP=[[] for i in range(COUNT)]
    while(l<r):
        if(am==1):
            if(ARCHIVE_FIT[l][f]>=turn):
                l+=1
            else:
                r-=1
                # swap fitness values of two particles in the archive
                for k in range(no_of_obj_func):
                    flagF[k]=ARCHIVE_FIT[l][k]
                    ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                    ARCHIVE_FIT[r][k]=flagF[k]
                #swap positions of two particles in the archive
                for k in range(COUNT):
                    flagP[k]=ARCHIVE_VAL[l][k]
                    ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                    ARCHIVE_VAL[r][k]=flagP[k]
                # swap their crowding distances
                temp=CrowdList[l]
                CrowdList[l]=CrowdList[r]
                CrowdList[r]=temp
                
        elif(am==2):
            if(ARCHIVE_FIT[l][f]<=turn):
                l+=1
            else:
                r-=1
                # swap fitness values of two particles in the archive
                for k in range(no_of_obj_func):
                    flagF[k]=ARCHIVE_FIT[l][k]
                    ARCHIVE_FIT[l][k]=ARCHIVE_FIT[r][k]
                    ARCHIVE_FIT[r][k]=flagF[k]
                #swap positions of two particles in the archive
                for k in range(COUNT):
                    flagP[k]=ARCHIVE_VAL[l][k]
                    ARCHIVE_VAL[l][k]=ARCHIVE_VAL[r][k]
                    ARCHIVE_VAL[r][k]=flagP[k]
                # swap their crowding distances
                temp=CrowdList[l]
                CrowdList[l]=CrowdList[r]
                CrowdList[r]=temp
                
            
                
#     l-=1
#     # swap fitness values of two particles in the archive
#     for k in range(no_of_obj_func):
#         flagF[k]=ARCHIVE_FIT[begin][k]
#         ARCHIVE_FIT[begin][k]=ARCHIVE_FIT[l][k]
#         ARCHIVE_FIT[l][k]=flagF[k]
#     # swap positions of two particles in the archive
#     for k in range(COUNT):
#         flagP[k]=ARCHIVE_VAL[begin][k]
#         ARCHIVE_VAL[begin][k]=ARCHIVE_VAL[l][k]
#         ARCHIVE_VAL[l][k]=flagP[k]
#     # swap their crowding distances
#     temp = CrowdList[begin];
#     CrowdList[begin] = CrowdList[l];
#     CrowdList[l] = temp;
    
#     if(l-begin>1):
#         lastpart=FITNESS_SORT.sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am)
#     if((lastpart-r)>1):
#         lastpart=FITNESS_SORT.sort_fitness(f,begin,lastpart,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL,CrowdList,am)
    return(lastpart)

In [ ]:
# FUNCTION TO CALCULATE THE INTERACTION SCORE OF EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
from itertools import combinations

def interaction_score(POPULATION_SIZE,no_of_pathways,SUM):
    DEG_OF_RELEVANCE=[random.uniform(0,1) for i in range(no_of_pathways)] # set degree of relevance of each pathway
    WEIGHTED_SUM=0.0
    WEIGHTED_VAL=0.0
    for i in range(no_of_pathways):
        SUM[i]=SUM[i]*DEG_OF_RELEVANCE[i]
        WEIGHTED_SUM+=SUM[i]
        WEIGHTED_VAL+=DEG_OF_RELEVANCE[i]
        
    return(WEIGHTED_SUM/WEIGHTED_VAL)
        
    
        
#if __name__=='__main__':
def Interaction_Score(POPULATION_SIZE):
    no_of_pathways=0
    NO_OF_GENES = []
    arr1=[]
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
                
    #print NO_OF_GENES
    arr2=[]
    arr3=[]
    UNI_GENES=[[[] for j in range(NO_OF_GENES[i])] for i in range(no_of_pathways)]
    i=0
    j=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/GENES_UNIPROTKB_ID.txt") as am:
        for line in am:
            arr2=line.split("\t\t")
            arr3=arr2[1].split("\n")
            UNI_GENES[i][j]=arr3[0]
            if(j<(NO_OF_GENES[i]-1)):
                i=i
                j=j+1
            elif(j==NO_OF_GENES[i]-1):
                i+=1
                j=0
                
    UNI_GENES_PAIR=[[] for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        UNI_GENES_PAIR[i]=list(combinations(UNI_GENES[i],2))
    NO_OF_PAIR=[(len(UNI_GENES_PAIR[i])) for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        for j in range(NO_OF_PAIR[i]):
            UNI_GENES_PAIR[i][j]=list(UNI_GENES_PAIR[i][j])
    #print (UNI_GENES_PAIR[0][0][1])
    
    count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/H_sapiens_interaction_output.txt") as am:
        for line in am:
            count+=1
    #print count
    
    ALL_GENES=[[] for i in range(count)]
    i=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/H_sapiens_interaction_output.txt") as am:
        for line in am:
            ALL_GENES[i]=line.split("\t")
            ALL_GENES[i][4]=ALL_GENES[i][4][:-2]
            i+=1
    #print ALL_GENES[10]
    
    SUM=[float(0.0) for i in range(no_of_pathways)]
    for i in range(no_of_pathways):
        for j in range(NO_OF_PAIR[i]):
            for k in range(count):
                if((UNI_GENES_PAIR[i][j][0]==ALL_GENES[k][0] and UNI_GENES_PAIR[i][j][1]==ALL_GENES[k][1]) | (UNI_GENES_PAIR[i][j][0]==ALL_GENES[k][1] and UNI_GENES_PAIR[i][j][1]==ALL_GENES[k][0])):
                    #print UNI_GENES_PAIR[i][j][0],"\t",UNI_GENES_PAIR[i][j][1],"\t",ALL_GENES[k][0],"\t",ALL_GENES[k][1]
                    SUM[i]+=float(ALL_GENES[k][4])
#     POPULATION_SIZE=10
    
    INTERACTION_SCORE=[[] for i in range(POPULATION_SIZE)]
    
    for i in range(POPULATION_SIZE):
        Sum=copy.copy(SUM)
        INTERACTION_SCORE[i]=interaction_score(POPULATION_SIZE,no_of_pathways,SUM)
        SUM=copy.copy(Sum)
#     print INTERACTION_SCORE
    return(INTERACTION_SCORE)

In [ ]:
# FUNCTION TO CALCULATE FITNESS VALUES OF PARTICLES IN POPULATION

import nbimporter
import random
import WEIGHTED_T_SCORE
import WEIGHTED_Z_SCORE
import INTERACTION_SCORE
reload(WEIGHTED_T_SCORE)
reload(WEIGHTED_Z_SCORE)
reload(INTERACTION_SCORE)


def update_particle(PARTICLE_FIT,no_of_obj_function,POP_SIZE):
    T_SCORE=WEIGHTED_T_SCORE.Weighted_T_Score(POP_SIZE) 
    Z_SCORE=WEIGHTED_Z_SCORE.Weighted_Z_Score(POP_SIZE)
    I_SCORE=INTERACTION_SCORE.Interaction_Score(POP_SIZE)
    for i in range(int(POP_SIZE)):
        PARTICLE_FIT[i][0]=T_SCORE[i]
        PARTICLE_FIT[i][1]=Z_SCORE[i]
        PARTICLE_FIT[i][2]=I_SCORE[i]
        
    return(PARTICLE_FIT)

In [ ]:
# FUNCTION TO INSERT NEW NON DOMINATED PARTICLES FROM POPULATION TO ARCHIVE

import random
from math import *
import numpy
import copy
import nbimporter
import CHECK_NONDOMINATED_SOLUTION
import CROWDING
reload(CROWDING)
reload(CHECK_NONDOMINATED_SOLUTION)




def update_archive(SWARM_SIZE,no_of_nondom_sol,archive_size,COUNT,ARCHIVE_VAL,PARTICLE_VAL,ARCHIVE_FIT,PARTICLE_FIT,no_of_obj_func):
    i=0
    for k in range(SWARM_SIZE):
        archive_temp=[int(no_of_nondom_sol) for j in range(1)]
        # if particle in population is non dominated 
        if(CHECK_NONDOMINATED_SOLUTION.check_nondom(k,archive_temp,no_of_obj_func,ARCHIVE_FIT,PARTICLE_FIT,COUNT,ARCHIVE_VAL)==1):
            no_of_nondom_sol=int(archive_temp[0])
            if(no_of_nondom_sol < archive_size): # if archive memory is not full insert particle
                i=no_of_nondom_sol 
                for j in range(COUNT):
                    ARCHIVE_VAL[i][j]=PARTICLE_VAL[k][j]
                for j in range(no_of_obj_func):
                    ARCHIVE_FIT[i][j]=PARTICLE_FIT[k][j]
                no_of_nondom_sol+=1
            else: # if memory is full , select particle to replace
                # compute crowding distances of particles in archive
                no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
                bottom=(no_of_nondom_sol-1)*0.90
                i=random.randint(bottom,no_of_nondom_sol-1)
                # insert new particles in archive
                for j in range(COUNT):
                    ARCHIVE_VAL[i][j]=PARTICLE_VAL[k][j]
                for j in range(no_of_obj_func):
                    ARCHIVE_FIT[i][j]=PARTICLE_FIT[i][j]
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO COMPUTE NEW VELOCITY AND POSITION FOR INDIVIDUAL PARTICLE IN POPULATION

import random
from math import *
import numpy
import copy

def update_velocity_position(no_of_nondom_sol,SWARM_SIZE,COUNT,VELOCITY,pBestPosition,PARTICLE_VAL,ARCHIVE_VAL):
    flag=(no_of_nondom_sol-1)*0.10
    for i in range(SWARM_SIZE):
        gBest=random.randint(0,int(flag))
        # compute new velocities for particles
        for j in range(COUNT):
            VELOCITY[i][j]=0.79*VELOCITY[i][j]+2.0*random.random()*(pBestPosition[i][j]-PARTICLE_VAL[i][j])+2.0*random.random()*(ARCHIVE_VAL[gBest][j]-PARTICLE_VAL[i][j])
    # compute new positions for each particle in the population
    for i in range(SWARM_SIZE):
        for j in range(COUNT):
            PARTICLE_VAL[i][j]=PARTICLE_VAL[i][j]+VELOCITY[i][j]
            
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO UPDATE PERSONAL BEST OF PARTICLES IN THE POPULATION

import random
from math import *
import numpy
import copy


def update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL):
    summ=0
    for i in range(SWARM_SIZE):
        summ=0 
        if((PARTICLE_FIT[i][0]<pBestFitness[i][0]) and (PARTICLE_FIT[i][1]<pBestFitness[i][1]) and (PARTICLE_FIT[i][2]>pBestFitness[i][2])):
            summ+=no_of_obj_func
        elif((PARTICLE_FIT[i][0]>pBestFitness[i][0]) and (PARTICLE_FIT[i][1]>pBestFitness[i][1]) and (PARTICLE_FIT[i][2]<pBestFitness[i][2])):
            summ=0
        else:
            summ=1
        if(summ==no_of_obj_func):
            measure=0
        else:
            if(summ==0):
                measure=1
            else:
                measure=random.randint(0,1)
        if(measure==0):
            for j in range(no_of_obj_func):
                pBestFitness[i][j]=PARTICLE_FIT[i][j]
            for j in range(COUNT):
                pBestPosition[i][j]=PARTICLE_VAL[i][j]

In [ ]:
# FUNCTION TO UPDATE PERSONAL BEST OF PARTICLES IN THE POPULATION

import random
from math import *
import numpy
import copy


def update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL):
    summ=0
    for i in range(SWARM_SIZE):
        summ=0 
        if((PARTICLE_FIT[i][0]<pBestFitness[i][0]) and (PARTICLE_FIT[i][1]<pBestFitness[i][1]) and (PARTICLE_FIT[i][2]>pBestFitness[i][2])):
            summ+=no_of_obj_func
        elif((PARTICLE_FIT[i][0]>pBestFitness[i][0]) and (PARTICLE_FIT[i][1]>pBestFitness[i][1]) and (PARTICLE_FIT[i][2]<pBestFitness[i][2])):
            summ=0
        else:
            summ=1
        if(summ==no_of_obj_func):
            measure=0
        else:
            if(summ==0):
                measure=1
            else:
                measure=random.randint(0,1)
        if(measure==0):
            for j in range(no_of_obj_func):
                pBestFitness[i][j]=PARTICLE_FIT[i][j]
            for j in range(COUNT):
                pBestPosition[i][j]=PARTICLE_VAL[i][j]

In [ ]:
# FUNCTION TO CALCULATE WEIGHTED T SCORE FOR EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
import numpy

def weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,normal_count,tumor_count,GENE_IN_PARTICLE):
    
    DEG_OF_RELEVANCE=[random.uniform(0,1) for i in range(GENE_IN_PARTICLE)]
    #print DEG_OF_RELEVANCE
    for i in range(GENE_IN_PARTICLE):
        Normal_Values[i]=(Normal_Values[i]*DEG_OF_RELEVANCE[i])
    for i in range(GENE_IN_PARTICLE):
        Tumor_Values[i]=(Tumor_Values[i]*DEG_OF_RELEVANCE[i])
    
    FINAL_VALUE1=0.0
    FINAL_WEIGHT=0.0
    FINAL_VALUE2=0.0
    
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE1+=Normal_Values[i]
        FINAL_WEIGHT+=DEG_OF_RELEVANCE[i]
    
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_VALUE2+=Tumor_Values[i]
           
    Mean1=FINAL_VALUE1/FINAL_WEIGHT # weighted mean of class 1
    Mean2=FINAL_VALUE2/FINAL_WEIGHT # weighted mean of class 2
   
    for i in range(GENE_IN_PARTICLE):
        N_Value[i]=(abs(N_Value[i]-Mean1)**2)*DEG_OF_RELEVANCE[i]
        
        
    for i in range(GENE_IN_PARTICLE):
        T_Value[i]=(abs(T_Value[i]-Mean2)**2)*DEG_OF_RELEVANCE[i]
        
    FINAL_STD1=0.0
    FINAL_STD2=0.0
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD1+=N_Value[i]
                               
    for i in range(GENE_IN_PARTICLE):
        FINAL_STD2+=T_Value[i]
        
    
    StdDev1=(((normal_count/(normal_count-1))*(FINAL_STD1/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 1
    StdDev2=(((tumor_count/(tumor_count-1))*(FINAL_STD2/FINAL_WEIGHT))**0.5) # weighted standard deviation of class 2
    
    WEIGHTED_T_SCORE=(abs(Mean1-Mean2))/(((StdDev1/normal_count)+(StdDev2/tumor_count))**0.5)
    return(WEIGHTED_T_SCORE)

def Weighted_T_Score(POPULATION_SIZE):
    
    no_of_pathways=0
    NO_OF_GENES = []
    NumberOfGenes=[]
    arr1=list()
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                NumberOfGenes.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
   

    NORMAL_DATA=[]
    TUMOR_DATA=[]
    n_count=0
    t_count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                n_count+=1
            elif(line.startswith("tumor")):
                t_count+=1
       
    COUNT=n_count+t_count
    ALL_GENES=[]
    Total=0
    VALUES=[]
    Normal_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    N_Value=[float(0.0) for i  in range(GENES_IN_PARTICLE)]
    Tumor_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    T_Value=[float(0.0)for i in range(GENES_IN_PARTICLE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    #print GENES           
    for i in range(no_of_pathways):
        for j in range(NumberOfGenes[i]):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                Normal_Values[g]+=float(VALUES[k])
                                N_Value[g]+=float(VALUES[k])
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                Tumor_Values[g]+=float(VALUES[k])
                                T_Value[g]+=float(VALUES[k])
            g+=1
            
    T_SCORE=[[] for i in range(POPULATION_SIZE)]
    for q in range(POPULATION_SIZE):
        #print ("Weighted T Score For Particle " +str(q+1)+" :")
        Norm_Values=copy.copy(Normal_Values)
        Tum_Values=copy.copy(Tumor_Values)
        N_Values=copy.copy(N_Value)
        T_Values=copy.copy(T_Value)
        
        T_SCORE[q]=weighted_t_score(Normal_Values,Tumor_Values,N_Value,T_Value,n_count,t_count,GENES_IN_PARTICLE)
        
        Normal_Values=copy.copy(Norm_Values)
        Tumor_Values=copy.copy(Tum_Values)
        N_Value=copy.copy(N_Values)
        T_Value=copy.copy(T_Values)
        
    return(T_SCORE)

In [ ]:
# FUNCTION TO CALCULATE WEIGHTED Z SCORE FOR EACH PARTICLE IN THE POPULATION

import random
from math import *
import copy
import numpy


def weighted_z_score(Exp_Values,GENES_IN_PARTICLE,Expression_Values,COUNT):
    WEIGHTED_RANDOM=[(random.random()) for i in range(GENES_IN_PARTICLE)]
    FINAL_WEIGHT=0.0
    for i in range(GENES_IN_PARTICLE):
        Exp_Values[i]=Exp_Values[i] * WEIGHTED_RANDOM[i]
    FINAL_P_ACT=0.0
    for i in range(GENES_IN_PARTICLE):
        FINAL_P_ACT+=Exp_Values[i]
        FINAL_WEIGHT+=WEIGHTED_RANDOM[i]
        
    PATHWAY_ACTIVITY=FINAL_P_ACT/(FINAL_WEIGHT**0.5)
    W_MEAN=FINAL_P_ACT/FINAL_WEIGHT 
    for i in range(GENES_IN_PARTICLE):
        Expression_Values[i]=(abs(Expression_Values[i]-W_MEAN)**2)*WEIGHTED_RANDOM[i]
    #print Expression_Values
    #print PATHWAY_ACTIVITY
    Variance=0.0
    for i in range(GENES_IN_PARTICLE):
        Variance+=Expression_Values[i]
        
    W_STD_DEV=(((COUNT/(COUNT-1))*(Variance/FINAL_WEIGHT))**0.5)
    WEIGHTED_Z_SCORE=(PATHWAY_ACTIVITY-W_MEAN)/W_STD_DEV
    return(WEIGHTED_Z_SCORE)


def Weighted_Z_Score(POPULATION_SIZE):
    no_of_pathways=0
    NO_OF_GENES = []
    NumberOfGenes=[]
    arr1=list()
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                NumberOfGenes.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
   

    NORMAL_DATA=[]
    TUMOR_DATA=[]
    n_count=0
    t_count=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                n_count+=1
            elif(line.startswith("tumor")):
                t_count+=1
       
    COUNT=n_count+t_count
    
    ALL_GENES=[]
    Total=0
    VALUES=[]
    Normal_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    N_Value=[float(0.0) for i  in range(GENES_IN_PARTICLE)]
    Tumor_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    T_Value=[float(0.0)for i in range(GENES_IN_PARTICLE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    #print GENES           
    for i in range(no_of_pathways):
        for j in range(NumberOfGenes[i]):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                Normal_Values[g]+=float(VALUES[k])
                                N_Value[g]+=float(VALUES[k])
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                Tumor_Values[g]+=float(VALUES[k])
                                T_Value[g]+=float(VALUES[k])
            g+=1
    Exp_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    Expression_Values=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    #print Normal_Values,"\n\n",Tumor_Values
    for i in range(GENES_IN_PARTICLE):
        Exp_Values[i]=Normal_Values[i]+Tumor_Values[i]
    Expression_Values=copy.copy(Exp_Values)
    #print len(Exp_Values)
    WEIGHTED_Z_SCORE=[[] for i in range(POPULATION_SIZE)]
    for p in range(POPULATION_SIZE):
        E1=copy.copy(Exp_Values)
        E2=copy.copy(Expression_Values)
        WEIGHTED_Z_SCORE[p]=weighted_z_score(Exp_Values,GENES_IN_PARTICLE,Expression_Values,COUNT)
        Exp_Values=copy.copy(E1)
        Expression_Values=copy.copy(E2) 
    return(WEIGHTED_Z_SCORE)

In [ ]:
# FUNCTION TO INSERT NONDOMINATED PARTICLES FROM POPULATION INTO ARCHIVE

import nbimporter
import random
from math import *
import copy
import numpy
import DELETE_PARTICLE
import CROWDING
reload(CROWDING)
reload(DELETE_PARTICLE)


def initialise_nondom_sol(no_of_nondom_sol,SWARM_SIZE,ARCHIVE_VAL,ARCHIVE_FIT,PARTICLE_VAL,PARTICLE_FIT,no_of_obj_func,archive_size,COUNT):
    flag=0
    for i in range(SWARM_SIZE):
        if(no_of_nondom_sol==0): # if archive is empty
            # insert nondominated solution from population into archive
            for j in range(COUNT):
                ARCHIVE_VAL[no_of_nondom_sol][j]=PARTICLE_VAL[i][j]
            for j in range(no_of_obj_func):
                ARCHIVE_FIT[no_of_nondom_sol][j]=PARTICLE_FIT[i][j]
            no_of_nondom_sol+=1
        else: # if archive is not empty
            flag=1
            for k in range(no_of_nondom_sol):
                total=0
                # check for non domination
                for j in range(no_of_obj_func):
                    if(((PARTICLE_FIT[i][j] < ARCHIVE_FIT[k][j]) and (j==0 | j==1)) or ((PARTICLE_FIT[i][j] > ARCHIVE_FIT[k][j]) and (j==2))):
                        total +=1;
                    if(total==no_of_obj_func): # if particle in population dominates
                    # delete particle in archive
                        no_of_nondom_sol=DELETE_PARTICLE.delete_particle(k,no_of_obj_func,no_of_nondom_sol,ARCHIVE_FIT)
                    elif(total==0): # particle in archive dominates
                        flag=0
                        break
            
            if(flag==1):
                if(no_of_nondom_sol < archive_size):# if archive memory is not full , insert particle
                    for j in range(COUNT):
                        ARCHIVE_VAL[no_of_nondom_sol][j]=PARTICLE_VAL[i][j]
                    for j in range(no_of_obj_func):
                        ARCHIVE_FIT[no_of_nondom_sol][j]=PARTICLE_FIT[i][j]
                    no_of_nondom_sol+=1
                else: # if archive memory is full , select particle to replace
                    # compute crowding distance values for particles in archive
                    no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
                    temp=(no_of_nondom_sol-1)*0.90
                    k=random.randint(int(temp),no_of_nondom_sol-1)
                    for j in range(COUNT):
                        ARCHIVE_VAL[k][j]=PARTICLE_VAL[i][j]
                    for j in range(no_of_obj_func):
                        ARCHIVE_FIT[k][j]=PARTICLE_FIT[i][j] 
            
    return(no_of_nondom_sol)

In [ ]:
# FUNCTION TO INITIALISE POSITION OF PARTICLES IN POPULATION

import random
from math import *
import numpy

def initialise_PARTICLE_VALUES(COUNT,SWARM_SIZE):
    xmin=-5.12
    xmax=5.12
    initial=[[abs(xmin+random.uniform(0,1)*(xmax-xmin)) for j in range(COUNT)] for i in range(SWARM_SIZE)]
    return(initial)

In [ ]:
# FUNCTION TO INITIALISE VELOCITY OF PARTICLES IN POPULATION

import random
from math import *
import numpy


def initialise_velocity(COUNT,SWARM_SIZE):
    xmin=-5.12
    xmax=5.12
    vmin=0.25*xmin
    vmax=0.25*xmax
    init=[[(vmin+random.uniform(0,1)*(vmax-vmin)) for j in range(COUNT)] for i in range(SWARM_SIZE)]
    return(init)

In [ ]:
# FUNCTION TO CALCULATE SENSITIVITY , SPECIFICITY , ACCURACY , F_SCORE AND AREA UNDER CURVE VALUES OF EACH PARTICLE IN THE POPULATION


import random
from math import *
import numpy
import copy


#if __name__=='__main__':
def values(POPULATION_SIZE,NONDOM_PARTICLE,temp_nondom,VALUES_1):
    temp=1
    no_of_pathways=0
    NO_OF_GENES = []
    arr1=[]
    GENES_IN_PARTICLE=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                no_of_pathways+=1
       
    GENES=[[] for j in range(no_of_pathways)]
    i=0
                
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/pathway/child_path(copy).txt") as br:
        for line in br:
            if(line.startswith("KEGG")):
                arr1=line.split('\t')
                GENES[i]=arr1[5].split(", ")
                NO_OF_GENES.append(len(GENES[i]))
                GENES_IN_PARTICLE+=len(GENES[i])
                i+=1
    ALL_GENES=[]
    Total=0
    VALUES=[]
    tempSENSITIVITY=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    tempSPECIFICITY=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    accuracy=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    f_score=[float(0.0) for i in range(GENES_IN_PARTICLE)]
    SENSITIVITY=[float(0.0) for i in range(POPULATION_SIZE)]
    SPECIFICITY=[float(0.0) for i in range(POPULATION_SIZE)]
    g=0
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as sn:
        for line in sn:
            if(line.startswith("class")):
                ALL_GENES=line.split("\t")
    tp=0.0
    tn=0.0
    fp=0.0
    fn=0.0
    for i in range(no_of_pathways):
        for j in range(int(NO_OF_GENES[i])):
            for k in range(len(ALL_GENES)):
                if(GENES[i][j].lower()==ALL_GENES[k]):
                    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as am:
                        for line in am:
                            if(line.startswith("normal")):
                                VALUES=line.split("\t")
                                if(VALUES[k]==str(0)):
                                    tn+=1.0
                                elif(VALUES[k]>str(0) and VALUES[k]<str(0.5)):
                                    fn+=1.0
                            elif(line.startswith("tumor")):
                                VALUES=line.split("\t")
                                if(VALUES[k]==str(1)):
                                    tp+=1.0
                                elif(VALUES[k]>str(0.5) and VALUES[k]<str(1.0)):
                                    fp+=1.0
                        tempSENSITIVITY[g]=(tp/(tp+fn))
                        tempSPECIFICITY[g]=(tn/(tn+fp))
                        accuracy[g]=(tp+tn)/(tp+tn+fp+fn)
                        f_score[g]=(2*tp)/((2*tp)+fp+fn)
                        tp=0.0
                        tn=0.0
                        fp=0.0
                        fn=0.0
                        g+=1
    
    
    
    Weighted_Score1=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score2=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score3=[float(0.0) for i in range(POPULATION_SIZE)]
    Weighted_Score4=[float(0.0) for i in range(POPULATION_SIZE)]
    
    SENSITIVITY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    ACCURACY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    F_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    
    MSENSITIVITY_SCORE=[int(1) for  i in range(POPULATION_SIZE)]
    # calculation of sensitivity of each particle in the population
    
    for  i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran1=random.random()
            SENSITIVITY_SCORE[i]+=(float(tempSENSITIVITY[j]*ran1))
            Weighted_Score1[i]+=ran1
            SENSITIVITY_SCORE[i]=(float(SENSITIVITY_SCORE[i])/float(Weighted_Score1[i]))
            MSENSITIVITY_SCORE[i]=MSENSITIVITY_SCORE[i]-SENSITIVITY_SCORE[i]
    SPECIFICITY_SCORE=[float(0.0) for i in range(POPULATION_SIZE)]
    MSPECIFICITY_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    # calculation of specificity of each particle in the population
    
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran2=random.random()
            SPECIFICITY_SCORE[i]+=(float(tempSPECIFICITY[j]*ran2))
            Weighted_Score2[i]+=ran2
            SPECIFICITY_SCORE[i]=(float(SPECIFICITY_SCORE[i])/float(Weighted_Score2[i]))
            MSPECIFICITY_SCORE[i]=MSPECIFICITY_SCORE[i]-SENSITIVITY_SCORE[i]
   
    MACCURACY_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of accuracy score of each particle in the population
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran3=random.random()
            ACCURACY_SCORE[i]+=(float(accuracy[j]*ran3))
            Weighted_Score3[i]+=ran3
            ACCURACY_SCORE[i]=(float(ACCURACY_SCORE[i])/float(Weighted_Score3[i]))
            MACCURACY_SCORE[i]=MACCURACY_SCORE[i]-ACCURACY_SCORE[i]

    MF_SCORE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of f score of each particle in the population
    for i in range(POPULATION_SIZE):
        for j in range(len(NO_OF_GENES)):
            ran4=random.random()
            F_SCORE[i]+=(float(f_score[j]*ran4))
            Weighted_Score4[i]+=ran4
            F_SCORE[i]=(float(F_SCORE[i])/float(Weighted_Score4[i]))
            MF_SCORE[i]=MF_SCORE[i]-F_SCORE[i]

    AREA_CURVE=[float(0.0) for i in range(POPULATION_SIZE)]
    MAREA_CURVE=[int(1) for i in range(POPULATION_SIZE)]
    
    # calculation of area under curve values of each particle in the population
    for i in range(POPULATION_SIZE):
        AREA_CURVE[i]=(SENSITIVITY_SCORE[i]/MSPECIFICITY_SCORE[i])
        MAREA_CURVE[i]=(MAREA_CURVE[i]-AREA_CURVE[i])
     
    for i in range(temp_nondom):
        VALUES_1[i][0]+=(MSENSITIVITY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][1]+=(MSPECIFICITY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][2]+=(MACCURACY_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][3]+=(MF_SCORE[int(NONDOM_PARTICLE[i])])
        VALUES_1[i][4]+=(MAREA_CURVE[int(NONDOM_PARTICLE[i])])

In [ ]:
# FUNCTION TO STORE PERSONAL BEST VALUES OF PARTICLES IN POPULATION 

import random
from math import *
import copy


def p_Best(pBestPosition,pBestFitness,PARTICLE_VAL,PARTICLE_FIT,SWARM_SIZE,no_of_obj_func,COUNT):
    for i in range(SWARM_SIZE): # store personal best positions of particles
        for j in range(COUNT):
            pBestPosition[i][j]=PARTICLE_VAL[i][j]
            
    for i in range(SWARM_SIZE): # store personal best fitness values of particles
        for j in range(no_of_obj_func):
            pBestFitness[i][j]=PARTICLE_FIT[i][j]

In [ ]:
#                  IMPLEMENTATION OF MULTI-OBJECTIVE PARTICLE SWARM OPTIMIZATION
#                  -------------------------------------------------------------                  
#
#
## This is the Main Function to execute the MOPSO Process


import nbimporter
import random
from math import *
import numpy
import copy
import init_VELOCITY as initvelo
import init_PARTICLE_VALUES as initparval
import PARTICLE_FITNESS
import store_pBest
import init_NONDOMINATED_SOLUTION
import CROWDING
import UPDATE_VELOCITY_POSITION
import UPDATE_ARCHIVE
import UPDATE_pBest
import sensitivity_specificity_accuracy_fscore_AUC
reload(sensitivity_specificity_accuracy_fscore_AUC)
reload(UPDATE_pBest)
reload(UPDATE_ARCHIVE)
reload(UPDATE_VELOCITY_POSITION)
reload(CROWDING)
reload(init_NONDOMINATED_SOLUTION)
reload(store_pBest)
reload(PARTICLE_FITNESS)
reload(initvelo)
reload(initparval)


# main function

if __name__=='__main__':
    normal_count=0         # Counts the number of Normal samples to be considered for individual gene
    tumor_count=0          # Counts the number of Tumor samples to be considered for individual gene
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                normal_count+=1
            elif(line.startswith("tumor")):
                tumor_count+=1
       
    COUNT=normal_count+tumor_count        # total number of samples for each gene
    SWARM_SIZE=25                         # total number of particles in the population
    no_of_obj_func=3                      # total number of objective functions considered
    Iteration=100
    archive_size=25                       # set maximum capacity of archive
    no_of_nondom_sol=0                    # Initial number of nondominated solutions in archive
    
    PARTICLE_VAL=initparval.initialise_PARTICLE_VALUES(COUNT,SWARM_SIZE) # initialise position values of particles in population 
    PARTICLE_FIT=[[[] for j in range(no_of_obj_func)] for i in range(SWARM_SIZE)]  # fitness values of particles in population
    
    ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]  #position values of particles in archive
    ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]  #fitness values of particles in archive
    
    VELOCITY=initvelo.initialise_velocity(COUNT,SWARM_SIZE)  # initialise velocity of particle in population
    
    pBestPosition=[[[]for j in range(COUNT)]for i in range(SWARM_SIZE)] # personal best position for each particle in population
    pBestFitness=[[[]for j in range(no_of_obj_func)]for i in range(SWARM_SIZE)]  # personal fitness for each particle in population
    
#     PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE) # update fitness values of each particle in population
    
    store_pBest.p_Best(pBestPosition,pBestFitness,PARTICLE_VAL,PARTICLE_FIT,SWARM_SIZE,no_of_obj_func,COUNT) # Store initial personal bests (both variable and fitness values) of particles
   
    
    # the output is written in the file MOPSO_OUTPUT.txt
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/MOPSO_OUTPUT.txt","w+") as am:
        for t in range(Iteration):
            # Compute new fitness values for each particle in the population
            PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE)
            # insert nondominated particles in archive from population
            no_of_nondom_sol=init_NONDOMINATED_SOLUTION.initialise_nondom_sol(no_of_nondom_sol,SWARM_SIZE,ARCHIVE_VAL,ARCHIVE_FIT,PARTICLE_VAL,PARTICLE_FIT,no_of_obj_func,archive_size,COUNT)
            
            if(no_of_nondom_sol>2):
                no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
            # Compute new velocity and position of each particle in the population
            no_of_nondom_sol=UPDATE_VELOCITY_POSITION.update_velocity_position(no_of_nondom_sol,SWARM_SIZE,COUNT,VELOCITY,pBestPosition,PARTICLE_VAL,ARCHIVE_VAL)

            # insert new non dominated particles in archive from population
            no_of_nondom_sol=UPDATE_ARCHIVE.update_archive(SWARM_SIZE,no_of_nondom_sol,archive_size,COUNT,ARCHIVE_VAL,PARTICLE_VAL,ARCHIVE_FIT,PARTICLE_FIT,no_of_obj_func)
            
            # update personal best for each particle in population
            UPDATE_pBest.update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL)
            
            NONDOM_PARTICLE=[int(0) for i in range(no_of_nondom_sol)]
            
            print (str(t))
            am.write("SIZE OF PARETO SET : ")
            am.write(str(no_of_nondom_sol))
            am.write("\n\n")
            am.write("ELEMENTS OF PARETO SET : ")
            for  i in range(no_of_nondom_sol):
                for j in range(SWARM_SIZE):
                    if(ARCHIVE_FIT[i][0]==PARTICLE_FIT[j][0]):
                        NONDOM_PARTICLE[i]=j
                        am.write(str(j+1))
                am.write("\t")
            am.write("\n\n")
                    
            am.write("FITNESS VALUES OF PARTICLES IN THE ARCHIVE : ")
            am.write("\n\n")
            am.write("WEIGHTED T SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][0]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("WEIGHTED Z SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][1]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("INTERACTION SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][2]))
                am.write("\t\t")
            am.write("\n")
            am.write("\n")
            am.write("**********************************************************************************************")    
            am.write("\n\n")
            
            temp_nondom=no_of_nondom_sol
            # reset number of nondominated particles, archive values and fitnesses to store values for next iteration
            no_of_nondom_sol=0
            
            ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]
            ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]
    
    # store the sensitivity , specificity , accuracy , f_score and area under curve value of the final non-dominated particles on archive
    VALUES_1=[[float(0.0) for j in range(5)] for i in range(temp_nondom)]
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/Output.txt" , "w+") as sn:
        sensitivity_specificity_accuracy_fscore_AUC.values(SWARM_SIZE,NONDOM_PARTICLE,temp_nondom,VALUES_1) # calculation of the corresponding values

        sn.write("ELEMENTS OF PARETO SET : ")
        for i in range(temp_nondom):
            sn.write(str(NONDOM_PARTICLE[i]+1))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SENSITIVITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][0]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SPECIFICITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][1]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("ACCURACY MEASURE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][2]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("F SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][3]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("AREA UNDER CURVE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][4]))
            sn.write("\t")
        sn.write("\n\n")

# PARTICLE SWARM OPTIMIZATION CODE2

In [ ]:
import random
import math
import matplotlib.pyplot as plt
#------------------------------------------------------------------------------
# TO CUSTOMIZE THIS PSO CODE TO SOLVE UNCONSTRAINED OPTIMIZATION PROBLEMS, CHANGE THE PARAMETERS IN THIS SECTION ONLY:
# THE FOLLOWING PARAMETERS MUST BE CHANGED.
def objective_function(x):
    y = 3*(1-x[0])**2*math.exp(-x[0]**2 - (x[1]+1)**2) - 10*(x[0]/5 - x[0]**3 - x[1]**5)*math.exp(-x[0]**2 - x[1]**2) -1/3*math.exp(-(x[0]+1)**2 - x[1]**2);
    return y
 
bounds=[(-3,3),(-3,3)]   # upper and lower bounds of variables
nv = 2                   # number of variables
mm = -1                   # if minimization problem, mm = -1; if maximization problem, mm = 1
 
# THE FOLLOWING PARAMETERS ARE OPTINAL.
particle_size=100         # number of particles
iterations=200           # max number of iterations
w=0.85                    # inertia constant
c1=1                    # cognative constant
c2=2                     # social constant
# END OF THE CUSTOMIZATION SECTION
#------------------------------------------------------------------------------    
class Particle:
    def __init__(self,bounds):
        self.particle_position=[]                     # particle position
        self.particle_velocity=[]                     # particle velocity
        self.local_best_particle_position=[]          # best position of the particle
        self.fitness_local_best_particle_position= initial_fitness  # initial objective function value of the best particle position
        self.fitness_particle_position=initial_fitness             # objective function value of the particle position
 
        for i in range(nv):
            self.particle_position.append(random.uniform(bounds[i][0],bounds[i][1])) # generate random initial position
            self.particle_velocity.append(random.uniform(-1,1)) # generate random initial velocity
 
    def evaluate(self,objective_function):
        self.fitness_particle_position=objective_function(self.particle_position)
        if mm == -1:
            if self.fitness_particle_position < self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
        if mm == 1:
            if self.fitness_particle_position > self.fitness_local_best_particle_position:
                self.local_best_particle_position=self.particle_position                  # update the local best
                self.fitness_local_best_particle_position=self.fitness_particle_position  # update the fitness of the local best
 
    def update_velocity(self,global_best_particle_position):
        for i in range(nv):
            r1=random.random()
            r2=random.random()
 
            cognitive_velocity = c1*r1*(self.local_best_particle_position[i] - self.particle_position[i])
            social_velocity = c2*r2*(global_best_particle_position[i] - self.particle_position[i])
            self.particle_velocity[i] = w*self.particle_velocity[i]+ cognitive_velocity + social_velocity
 
    def update_position(self,bounds):
        for i in range(nv):
            self.particle_position[i]=self.particle_position[i]+self.particle_velocity[i]
 
            # check and repair to satisfy the upper bounds
            if self.particle_position[i]>bounds[i][1]:
                self.particle_position[i]=bounds[i][1]
            # check and repair to satisfy the lower bounds
            if self.particle_position[i] < bounds[i][0]:
                self.particle_position[i]=bounds[i][0]
                 
class PSO():
    def __init__(self,objective_function,bounds,particle_size,iterations):
 
        fitness_global_best_particle_position=initial_fitness
        global_best_particle_position=[]
 
        swarm_particle=[]
        for i in range(particle_size):
            swarm_particle.append(Particle(bounds))
        A=[]
         
        for i in range(iterations):
            for j in range(particle_size):
                swarm_particle[j].evaluate(objective_function)
 
                if mm ==-1:
                    if swarm_particle[j].fitness_particle_position < fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
                if mm ==1:
                    if swarm_particle[j].fitness_particle_position > fitness_global_best_particle_position:
                        global_best_particle_position = list(swarm_particle[j].particle_position)
                        fitness_global_best_particle_position = float(swarm_particle[j].fitness_particle_position)
            for j in range(particle_size):
                swarm_particle[j].update_velocity(global_best_particle_position)
                swarm_particle[j].update_position(bounds)
                 
            A.append(fitness_global_best_particle_position) # record the best fitness
             
             
        print('Optimal solution:', global_best_particle_position)
        print('Objective function value:', fitness_global_best_particle_position)
        print('Evolutionary process of the objective function value:')
        plt.plot(A)
#------------------------------------------------------------------------------
if mm == -1:
    initial_fitness = float("inf") # for minimization problem
if mm == 1:
    initial_fitness = -float("inf") # for maximization problem
#------------------------------------------------------------------------------   
# Main PSO         
PSO(objective_function,bounds,particle_size,iterations)